# N-glycosilation prediciton model

The model is a fine-tuned ESM2 model aimed to predict resiudes undergo N-glycosilation based on antigen protein sequence

The MIT License (MIT) Copyright (c) 2016 AYLIEN Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


## Requirements

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TORCH_HOME'] = "../torch_hub"

import pandas as pd
import numpy  as np

from copy import copy
from Bio import SeqIO

import torch
from torch.utils.data import Dataset
from torch import nn

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction

import esm
from esm.pretrained import load_model_and_alphabet_hub

## Set model

In [ ]:
class ESM2ForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2):
        super().__init__()
        self.num_labels = num_labels    
        self.model_name = "esm2_t33_650M_UR50D"
        self.esm2, self.esm2_alphabet = load_model_and_alphabet_hub(self.model_name)         
        self.classifier = nn.Linear(1280, self.num_labels)

    def forward(self, token_ids):
        outputs = self.esm2.forward(token_ids, repr_layers=[33])['representations'][33]
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)
        return SequenceClassifierOutput(logits=logits)
    
class ESM2Dataset(Dataset):
    """
    A class to represent a sutable data set for model. 
    
    convert original pandas data frame to model set,
    where 'token ids' is ESM2 embedings corresponed to protein sequence 
    and 'lables' is a contact number values
    Attributes:
        df (pandas.DataFrame): dataframe with two columns: 
                0 -- preotein sequence in string ('GLVM') or list (['G', 'L', 'V', 'M']) format
                1 -- N-glycosilation label [0, 0, 1, -100, 0] format
        esm2_batch_converter (function):
                    ESM function callable to convert an unprocessed (labels + strings) batch to a
                    processed (labels + tensor) batch.
    """
    def __init__(self, df):
        self.df = df
        _, esm2_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
        self.esm2_batch_converter = esm2_alphabet.get_batch_converter()

    def __getitem__(self, idx):
        item = {}
        _, _, esm2_batch_tokens = self.esm2_batch_converter([('' , ''.join(self.df.iloc[idx,0]))])
        item['labels'] = torch.unsqueeze(torch.LongTensor(self.df.iloc[idx, 1]), 0)
        item['token_ids'] = esm2_batch_tokens
        print(item)
        return item

    def __len__(self):
        return len(self.df)

In [ ]:
model = ESM2ForTokenClassification().cuda()

## Prediction

In [ ]:
model.load_state_dict(torch.load('../models/sema2.0_esm2_ptm.pth'))
model.eval()
model.cuda()

In [ ]:
## upload example test set
# test_df = pd.read_pickle('../data/Nglyco_test.pkl')
# test_ds = ESM2Dataset(test_df[['seq', 'balanced_labels']])

In [ ]:
## or use simple example
test_ds = ESM2Dataset(pd.DataFrame({'seq': ['AQIGPVTDLHITNANISPDGFSRPAVLAGGTFPGPTIAGNTGDNFQITVFNDLTDPSMLTDTSIHWHGLFQKGTNWADGPAFVTQCPIITGQSFDYNFNVPGQAGTFWYHSHLSTQYCDGLRGPFVVYDPNDPNASLYDVDDDTTIITLADWYHTLAQQEPIGAAITADATLINGLGRSFTNTTASPLSVITVQSGKRYRMRLVSISCDPNYLFSIDGHDMTIIEVDGVNSQQLTVDQIQIFAAQRYSFVLNANQPVGNYWIRAQPNSGGQGFDGGINSAILRYEGATVEDPTTTAPTTFSNPLVETDLHPLADLGVPGQPFRGGADDPLVLNLAFANGRFSIDGVSFVPPTVPVLLQILSGAQNAQDLLPAGSVISLPSNSVIEVALPAGAAGGPHPFHLHGHNFAVVQSANNATPNYVNPIWRDTVSIGGTGDNVTIRFTTNNPGPWFLHCHIDWHLEAGFAIVFAEDIPDTASANPVPQAWSDLCPAYDQAHNIS'],
                                   'label': [0]}))

In [ ]:
with torch.no_grad():
    for it in test_ds:
        pred = model.forward(it['token_ids'].cuda())[0][0].cpu().numpy()

In [ ]:
res = np.argmax(pred, axis=1)
print(res)